In [1]:
! pip install --quiet cfgrib xarray pandas pyarrow

In [3]:
endpoint = 'http://172.200.202.2:80'
access_key = 'O6UZCRYPS9OPWUD1DYF8'
secret_key = 'HnF34+7JkYFusmbU++Cv0/YLUiEpWGBRZTjmUsuu'
bucket_name='csnowdb'
schema_name='grib2'

# Query

In [4]:
import io
import os
import pyarrow as pa
from pyarrow import csv as pa_csv
import pyarrow.parquet as pq
from io import StringIO
import numpy as np
import pandas as pd
import vastdb

def read_parquet(file_path):
    """Reads Parquet data from a file."""
    try:
        return pq.read_table(file_path)
    except Exception as e:
        raise RuntimeError(f"Error reading Parquet file: {e}") from e

def connect_to_vastdb(endpoint, access_key, secret_key):
    """Connects to VastDB."""
    try:
        session = vastdb.connect(endpoint=endpoint, access=access_key, secret=secret_key)
        print("Connected to VastDB")
        return session
    except Exception as e:
        raise RuntimeError(f"Failed to connect to VastDB: {e}") from e

def query_vastdb(session, bucket_name, schema_name, table_name):
    """Writes data to VastDB."""
    with session.transaction() as tx:
        bucket = tx.bucket(bucket_name)
        schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
        table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

        return table.select().read_all()

In [5]:
session = connect_to_vastdb(endpoint, access_key, secret_key)

Connected to VastDB


## Query Meta

In [8]:
table_name='grib2_data_ingestion_log'

with session.transaction() as tx:
    bucket = tx.bucket(bucket_name)
    schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
    table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

    df = table.select(limit_rows=10).read_all().to_pandas()
df

,ingestion_id,source_url,filename,file_size_bytes,download_timestamp,processing_timestamp,ingestion_timestamp,record_count,source_name,download_duration_seconds,processing_duration_seconds,grib2_variables,ingestion_batch_id,status
0,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,1271243,2025-06-04 11:46:18.714024,2025-06-04 11:46:21.514411,2025-06-04 11:46:28.294067,2882400,GEM Global 15km,0.674091,3.227568,['absv'],GEM Global 15km_batch_1_1749037578,SUCCESS
1,CMC_glb_ABSV_ISBL_250_latlon.15x.15_2025060400...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_ABSV_ISBL_250_latlon.15x.15_2025060400...,1366582,2025-06-04 11:46:18.715355,2025-06-04 11:46:28.307415,2025-06-04 11:46:34.087987,2882400,GEM Global 15km,0.700882,2.167939,['absv'],GEM Global 15km_batch_1_1749037578,SUCCESS
2,CMC_glb_ABSV_ISBL_700_latlon.15x.15_2025060400...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_ABSV_ISBL_700_latlon.15x.15_2025060400...,1740084,2025-06-04 11:46:19.416390,2025-06-04 11:46:34.098587,2025-06-04 11:46:39.806233,2882400,GEM Global 15km,0.720940,2.152429,['absv'],GEM Global 15km_batch_1_1749037578,SUCCESS
3,CMC_glb_ABSV_ISBL_500_latlon.15x.15_2025060400...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_ABSV_ISBL_500_latlon.15x.15_2025060400...,1545074,2025-06-04 11:46:19.388333,2025-06-04 11:46:41.026413,2025-06-04 11:46:46.677081,2882400,GEM Global 15km,1.003888,2.082892,['absv'],GEM Global 15km_batch_1_1749037578,SUCCESS
4,CMC_glb_ABSV_ISBL_850_latlon.15x.15_2025060400...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_ABSV_ISBL_850_latlon.15x.15_2025060400...,1700124,2025-06-04 11:46:20.137598,2025-06-04 11:46:46.691808,2025-06-04 11:46:52.515234,2882400,GEM Global 15km,1.376216,2.281382,['absv'],GEM Global 15km_batch_1_1749037578,SUCCESS
5,CMC_glb_ACPCP_SFC_0_latlon.15x.15_2025060400_P...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_ACPCP_SFC_0_latlon.15x.15_2025060400_P...,205,2025-06-04 11:46:52.527585,2025-06-04 11:46:55.317995,2025-06-04 11:47:00.691980,2882400,GEM Global 15km,0.221725,1.769324,['acpcp'],GEM Global 15km_batch_2_1749037612,SUCCESS
6,CMC_glb_ALBDO_SFC_0_latlon.15x.15_2025060400_P...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_ALBDO_SFC_0_latlon.15x.15_2025060400_P...,250134,2025-06-04 11:46:52.528753,2025-06-04 11:47:00.702824,2025-06-04 11:47:06.157795,2882400,GEM Global 15km,0.493053,1.878707,['al'],GEM Global 15km_batch_2_1749037612,SUCCESS
7,CMC_glb_CIN_SFC_0_latlon.15x.15_2025060400_P00...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_CIN_SFC_0_latlon.15x.15_2025060400_P00...,3162627,2025-06-04 11:46:53.021985,2025-06-04 11:47:06.170397,2025-06-04 11:47:12.698703,2882400,GEM Global 15km,1.007277,2.941301,['unknown'],GEM Global 15km_batch_2_1749037612,SUCCESS
8,CMC_glb_CWAT_EATM_0_latlon.15x.15_2025060400_P...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_CWAT_EATM_0_latlon.15x.15_2025060400_P...,560339,2025-06-04 11:46:54.029525,2025-06-04 11:47:12.710439,2025-06-04 11:47:18.883252,2882400,GEM Global 15km,0.524039,2.402540,['cwat'],GEM Global 15km_batch_2_1749037612,SUCCESS
9,CMC_glb_CAPE_SFC_0_latlon.15x.15_2025060400_P0...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_CAPE_SFC_0_latlon.15x.15_2025060400_P0...,4348478,2025-06-04 11:46:52.749469,2025-06-04 11:47:18.899190,2025-06-04 11:47:25.325868,2882400,GEM Global 15km,2.568076,2.837856,['unknown'],GEM Global 15km_batch_2_1749037612,SUCCESS


## Query Data

In [11]:
table_name='grib2_data'

with session.transaction() as tx:
    bucket = tx.bucket(bucket_name)
    schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
    table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

    df = table.select(limit_rows=10).read_all().to_pandas()
df

,latitude,longitude,time,step,isobaricInhPa,valid_time,absv,source_file,ingestion_timestamp,acpcp,...,heightAboveGround,strd,nominalTop,tisr,d2m,ssrd,fg10,min_i10fg,i10fg,gh
0,-57.3,-16.80,2025-06-04,0.0,200.0,2025-06-04,-0.000145,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-57.3,-16.65,2025-06-04,0.0,200.0,2025-06-04,-0.000144,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-57.3,-16.50,2025-06-04,0.0,200.0,2025-06-04,-0.000143,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-57.3,-16.35,2025-06-04,0.0,200.0,2025-06-04,-0.000142,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-57.3,-16.20,2025-06-04,0.0,200.0,2025-06-04,-0.000141,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-57.3,-16.05,2025-06-04,0.0,200.0,2025-06-04,-0.000140,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-57.3,-15.90,2025-06-04,0.0,200.0,2025-06-04,-0.000138,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-57.3,-15.75,2025-06-04,0.0,200.0,2025-06-04,-0.000136,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-57.3,-15.60,2025-06-04,0.0,200.0,2025-06-04,-0.000134,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-57.3,-15.45,2025-06-04,0.0,200.0,2025-06-04,-0.000131,CMC_glb_ABSV_ISBL_200_latlon.15x.15_2025060400...,2025-06-04 11:46:23.934558,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Predicate pushdown

In [13]:
from ibis import _

predicate=(_.step != 0.0)

In [ ]:
table_name='grib2_data'

with session.transaction() as tx:
    bucket = tx.bucket(bucket_name)
    schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
    table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

    df = table.select(predicate=predicate, limit_rows=1000).read_all().to_pandas()
df